<a href="https://colab.research.google.com/github/Taisiia2001/data-science-portfolio/blob/main/Non_image_EEG_data_classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [30]:
#load EEG dataset
data = pd.read_csv("Epileptic Seizure Recognition.csv")

In [31]:
#check for missing or invalid values
missing_values = data.isnull().sum()
invalid_values = (data == np.inf).sum()
print ("Missing values:\n", missing_values)
print ("Invalid values:\n", invalid_values)

Missing values:
 Unnamed    0
X1         0
X2         0
X3         0
X4         0
          ..
X175       0
X176       0
X177       0
X178       0
y          0
Length: 180, dtype: int64
Invalid values:
 Unnamed    0
X1         0
X2         0
X3         0
X4         0
          ..
X175       0
X176       0
X177       0
X178       0
y          0
Length: 180, dtype: int64


In [32]:
eeg_x = data.drop(["Unnamed", "y"], axis = 1)
eeg_y = data["y"]


In [33]:
eeg_x.shape

(11500, 178)

In [34]:
eeg_y.shape

(11500,)

In [35]:
eeg_data = eeg_x.values
np.save("eeg_data.npy", eeg_data)
eeg_labels = eeg_y.values
np.save("eeg_labels.npy", eeg_labels )

In [36]:
#load your EEG data
X = np.load("eeg_data.npy")
y = np.load("eeg_labels.npy")

In [37]:
#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
#reshape the data to fit the input shape of the CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape (X_test, (X_test.shape[0], X_test.shape[1], 1))

In [39]:
y_train_uq = np.unique(y_train)
print (y_train_uq)

y_test_uq = np.unique(y_test)
print (y_test_uq)


[1 2 3 4 5]
[1 2 3 4 5]


In [40]:
#convert labels to one-hot encoded vectors
y_train = to_categorical(y_train-1, num_classes=5)
y_test = to_categorical(y_test-1, num_classes=5)


In [41]:
y_train.shape

(9200, 5)

In [42]:
y_test.shape

(2300, 5)

In [43]:
X_train.shape

(9200, 178, 1)

In [ ]:
X_test.shape

(2300, 178, 1)

In [44]:
#define the CNN architecture
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation="relu", input_shape=(X_train.shape[1],1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))


In [45]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 176, 32)           128       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 88, 32)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 86, 64)            6208      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 43, 64)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2752)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [47]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=14, batch_size=32)


Epoch 1/14
288/288 [==============================] - 7s 20ms/step - loss: 2.3015 - accuracy: 0.2640 - val_loss: 1.3191 - val_accuracy: 0.3843
Epoch 2/14
288/288 [==============================] - 6s 19ms/step - loss: 1.3853 - accuracy: 0.3413 - val_loss: 1.1968 - val_accuracy: 0.5270
Epoch 3/14
288/288 [==============================] - 7s 24ms/step - loss: 1.2728 - accuracy: 0.4473 - val_loss: 1.0722 - val_accuracy: 0.5457
Epoch 4/14
288/288 [==============================] - 6s 19ms/step - loss: 1.0793 - accuracy: 0.5426 - val_loss: 0.9567 - val_accuracy: 0.5843
Epoch 5/14
288/288 [==============================] - 7s 25ms/step - loss: 0.9375 - accuracy: 0.5949 - val_loss: 0.8311 - val_accuracy: 0.5978
Epoch 6/14
288/288 [==============================] - 6s 19ms/step - loss: 0.8255 - accuracy: 0.6271 - val_loss: 0.7774 - val_accuracy: 0.6465
Epoch 7/14
288/288 [==============================] - 8s 28ms/step - loss: 0.7657 - accuracy: 0.6529 - val_loss: 0.7917 - val_accuracy: 0.6339

In [48]:
# Evaluate the model on the test set
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

72/72 [==============================] - 1s 9ms/step - loss: 0.5985 - accuracy: 0.7248
Test loss: 0.5985015630722046
Test accuracy: 0.7247825860977173


In [49]:
# Create a plot of accuracy and loss over time
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='Training Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'), row=1, col=2)
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'), row=1, col=2)

fig.update_layout(height=500, width=1000, title='Model Performance Metrics')
fig.show()